# get_data
> a series of functions and classes to simplify asynchronous and synchronous API requests

In [ ]:
# | default_exp Transport


In [ ]:
# | hide
from nbdev.showdoc import show_doc


In [ ]:
# | export
from fastcore.basics import patch_to
from fastcore.test import test_eq

from dataclasses import dataclass, field
from enum import Enum

from abc import abstractmethod

from typing import Optional, Union

from requests_toolbelt.utils import dump
import json

import requests

import aiohttp
import asyncio

from nbdev_domo.ResponseGetData import ResponseGetData


# Transport Methods by Library

This code base supports two API request libraries, `requests.request` (synchronous) and `aiohttp.ClientRequest` (asynchronous) this can be extended as new libraries emerge with different performance characteristics.

In [ ]:
# | exporti
class RequestTransport:
    """
    The `RequestTransport` abstract base class adds consistency to the transport methods with consistent methods for 
    generating headers, as well as generating requests from APIs.
    """

    def __init__(self, auth_header: Optional[dict] = None,  # optional API authentication header
                 # defalt timeout to prevent infinite loops
                 request_timeout: Optional[int] = 10
                 ):

        self.auth_header = auth_header
        self.request_timeout = request_timeout

    @abstractmethod
    def _request() -> ResponseGetData:
        """Because each library has their own request methods, _request will be implemented in the interface classes of RequestTransport"""
        pass

    def dump_response(self, response):
        data = dump.dump_all(response)
        return str(data.decode('utf-8'))

    @staticmethod
    def _obj_to_json(obj):
        return json.dumps(obj, default=str)

    def _headers_default_receive_json(self):
        headers = {'Accept': 'application/json'}
        if self.auth_header:
            headers.update(self.auth_header)
        return headers

    def _headers_receive_csv(self):
        headers = self._headers_default_receive_json()
        headers['Accept'] = 'text/csv'
        return headers

    def _headers_send_json(self):
        headers = self._headers_default_receive_json()
        headers['Content-Type'] = 'application/json'
        return headers

    def _headers_send_csv(self):
        headers = self._headers_default_receive_json()
        headers['Content-Type'] = 'text/csv'
        return headers

    def _headers_send_gzip(self):
        headers = self._headers_default_receive_json()
        headers['Content-Type'] = 'text/csv'
        headers['Content-Encoding'] = 'gzip'
        return headers


In [ ]:
# if users pass auth_header to RequestTransport will be added to all auth requests

rt = RequestTransport(auth_header={'x-domo-authentication': '123'})
headers = rt._headers_default_receive_json()

# validate headers include authentication header
test_eq(headers.keys(), ['Accept', 'x-domo-authentication'])

# validate timeout defaults to 10 seconds
test_eq(rt.request_timeout, 10)


In [ ]:
# | exporti
class HTTPMethod(Enum):
    """utility class used by RequestTransport to enumerate HTTP methods"""

    GET = 'GET'
    POST = 'POST'
    PUT = 'PUT'
    PATCH = 'PATCH'
    DELETE = 'DELETE'


In [ ]:
# | exporti
# Each method establishes the appropriate headers before calling the request method.


@patch_to(RequestTransport)
def get(self, url, params=None, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None):
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_default_receive_json()
    return self._request(url, HTTPMethod.GET, headers, params, session=session)


@patch_to(RequestTransport)
def get_csv(self, url, params=None, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None):
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_receive_csv()
    return self._request(url, HTTPMethod.GET, headers, params, session=session)


@patch_to(RequestTransport)
def post(self, url, body, params=None, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None) -> ResponseGetData:
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_send_json()
    return self._request(url, HTTPMethod.POST, headers, params,
                         self._obj_to_json(body), session=session)


@patch_to(RequestTransport)
def put(self, url, body, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None):
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_send_json()
    return self._request(url, HTTPMethod.PUT, headers, {},
                         self._obj_to_json(body), session=session)


@patch_to(RequestTransport)
def put_csv(self, url, body, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None):
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_send_csv()
    return self._request(url, HTTPMethod.PUT, headers, {}, body, session=session)


@patch_to(RequestTransport)
def put_gzip(self, url, body, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None):
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_send_gzip()
    return self._request(url, HTTPMethod.PUT, headers, {}, body, session=session)


@patch_to(RequestTransport)
def patch(self, url, body, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None):
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_send_json()
    return self._request(url, HTTPMethod.PATCH, headers, {},
                         self._obj_to_json(body), session=session)


@patch_to(RequestTransport)
def delete(self, url, request_timeout: Optional[int] = None, session: Optional[aiohttp.ClientSession] = None):
    if request_timeout:
        self.request_timeout = request_timeout

    headers = self._headers_default_receive_json()
    return self._request(url, HTTPMethod.DELETE, headers, session=session)


## Requests.request - for synchronous code execution
The `TransportSync` class is a simple wrapper for `requests.request`

In [ ]:
# | export
class TransportSync(RequestTransport):
    def __init__(self, auth_header: Optional[dict] = None,  # for API Authentication
                 request_timeout: int = 10  # for default timeout to prevent infinite loops
                 ):
        super().__init__(auth_header=auth_header, request_timeout=request_timeout)

    def _request(self,
                 url: str,
                 method: HTTPMethod,
                 headers: dict,
                 params: dict = field(default_factory=dict),
                 body: Union[str, dict, None] = None, **kwargs
                 ):

        # self.logger.debug('{} {} {}'.format(method, url, body))

        request_args = {'method': method.value,
                        'url': url,
                        'headers': headers,
                        'params': params,
                        'data': body,
                        'stream': True}

        if self.request_timeout:
            request_args['timeout'] = self.request_timeout

        res = requests.request(**request_args)
        return ResponseGetData._from_requests_response(res=res, auth_header=self.auth_header)


#### Sample implementation of TransportSync

In [ ]:
ts = TransportSync(auth_header={'x-domo-authentication': '123'})

test_eq(isinstance(ts, TransportSync), True)

get_res = ts.get(
    url='http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita')

# test for a 200 status
test_eq(get_res.status, 200)

# validate that the response object includes the sent headers
test_eq(get_res.auth_header.keys(), ['x-domo-authentication'])


## Asyncio.session - for asynchronous code execution

The `TransportAsync` class is a wrapper for `aiohttp`'s ClientSession and ClientResponse implementations.  Notice the use of ```async/await``` in the code samples


In [ ]:
# | export
class TransportAsync(RequestTransport):
    """wrapper for aiohttp.ClientSession and aiohttp.ClientResponse for handling asynchronous code execution.  In the event of request_timeout, will default to synchronous code execution via requests.request library"""

    def __init__(self,
                 # API Authentication header
                 auth_header: Optional[dict] = None,
                 request_timeout: int = 10,  # request timeout to prevent infinite loops
                 session: Optional[aiohttp.ClientSession] = None
                 ):

        self.session = session
        super().__init__(auth_header=auth_header, request_timeout=request_timeout)

    async def _request(self,
                       url: str,
                       method: HTTPMethod,
                       headers: dict,
                       params: dict = field(default_factory=dict),
                       body: Union[str, dict, None] = None,
                       session: Optional[aiohttp.ClientSession] = None,
                       ):

        session = session or self.session

        if session is None:
            is_close_session = True
            session = aiohttp.ClientSession()
        else:
            is_close_session = False

        # self.logger.debug('{} {} {}'.format(method, url, body))

        request_args = {'url': url,
                        'headers': headers,
                        'params': params,
                        'data': body}

        try:
            res = await getattr(session, method.value.lower())(
                timeout=aiohttp.ClientTimeout(total=self.request_timeout),
                **request_args)

            return await ResponseGetData._from_aiohttp_response(res, auth_header=self.auth_header)

        except asyncio.TimeoutError as e:
            print('defaulting to sync request.  TimeoutError')

            res = requests.request(
                method=method.value,
                timeout=self.request_timeout,
                **request_args)

            return ResponseGetData._from_requests_response(res, auth_header=self.auth_header)

        finally:
            if is_close_session:
                await session.close()


#### Basic implementation of TransportAsync

In [ ]:
tas = TransportAsync(auth_header={'x-domo-authentication': '123'})
test_eq(isinstance(tas, TransportAsync), True)

get_res = await tas.get(url='http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita', request_timeout=10)

get_res.response.get('drinks')[0].get('idDrink')

# test for a 200 status
test_eq(get_res.status, 200)

# validate that the response object includes the sent headers
test_eq(get_res.auth_header.keys(), ['x-domo-authentication'])

get_res.response.get('drinks')[0].get('idDrink')


'11007'

#### Sample implementation of TransportAsync with loop
To LOOP over asynchronous code blocks, take advantage of `asyncio.gather()`

1. Notice how the ```get_drink()``` function implements what would usually be contained within the FOR LOOP allowing us to implement a list comprehension.  The ```asyncio.sleep()``` function highlights the fact that we are retrieving all the requests simultaneously.

2. Use ```await asyncio.gather( * [list_of_asynchronous_functions])``` forces the code to wait till all the APIs have returned a response before proceeding to the final step of printing the results.

3. Each ```.get()``` request returns a `ResponseGetData` object.  We can test ```iif(ResponseGetData.is_success)``` to validate having received a 200 response.  The response contenent will always be in the .response attribute.

In [ ]:
async def get_drink(drink_name: str):
    url = f'http://www.thecocktaildb.com/api/json/v1/1/search.php?s={drink_name}'
    print(f'getting - "{url}"')

    await(asyncio.sleep(2))
    return await tas.get(url=url)

drink_names = ['margarita', 'old fashioned',
               'gin and tonic', 'screwdriver', 'vodka tonic']

drinks_requests = await asyncio.gather(
    * [get_drink(drink_name) for drink_name in drink_names]
)

for drink in drinks_requests:
    [print(drink_match.get('idDrink'))
     for drink_match in drink.response.get('drinks') if drink.is_success]


getting - "http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita"
getting - "http://www.thecocktaildb.com/api/json/v1/1/search.php?s=old fashioned"
getting - "http://www.thecocktaildb.com/api/json/v1/1/search.php?s=gin and tonic"
getting - "http://www.thecocktaildb.com/api/json/v1/1/search.php?s=screwdriver"
getting - "http://www.thecocktaildb.com/api/json/v1/1/search.php?s=vodka tonic"
11007
11118
17216
16158
12322
178332
11001
11403
12162
12091
178364
